# VERİ DÜZENLEME

In [68]:
import pandas as pd
import polars as pl
import chess.pgn
import chess

In [70]:
nakamura=pl.read_parquet("Nakamura.parquet")

In [69]:
carlsen=pl.read_parquet("Carlsen.parquet")

In [71]:
caruana=pl.read_parquet("Caruana.parquet")

## Hamle Sayısı Değişkeni

In [72]:
def calculate_number_of_moves(evaluations):
    length = len(evaluations)
    if length % 2 == 0:
        return length // 2
    else:
        return (length + 1) // 2

In [73]:
carlsen = carlsen.with_columns(
    pl.col("Evaluations").map_elements(calculate_number_of_moves, return_dtype=pl.Int64).alias("Number of Moves")
)

In [74]:
nakamura = nakamura.with_columns(
    pl.col("Evaluations").map_elements(calculate_number_of_moves, return_dtype=pl.Int64).alias("Number of Moves")
)

In [75]:
caruana = caruana.with_columns(
    pl.col("Evaluations").map_elements(calculate_number_of_moves, return_dtype=pl.Int64).alias("Number of Moves")
)

## Hamle Sayısına Göre Sınıflandırma

In [76]:
def determine_game_length(num_moves):
    if num_moves < 20:
        return "miniature"
    elif 20 <= num_moves < 40:
        return "medium-length"
    elif 40 <= num_moves < 60:
        return "long"
    else:
        return "very long"

In [77]:
carlsen = carlsen.with_columns(
    pl.col("Number of Moves").map_elements(determine_game_length, return_dtype=pl.Utf8).alias("Game Length")
)

In [78]:
nakamura = nakamura.with_columns(
    pl.col("Number of Moves").map_elements(determine_game_length, return_dtype=pl.Utf8).alias("Game Length")
)

In [79]:
caruana = caruana.with_columns(
    pl.col("Number of Moves").map_elements(determine_game_length, return_dtype=pl.Utf8).alias("Game Length")
)

## En İyi Hamle Oranının Sınıflandırılması

In [80]:
def classify_best_move_rate(rate):
    rate_percentage = rate * 100
    if rate_percentage < 30:
        return "kötü"
    elif 30 <= rate_percentage < 60:
        return "orta"
    elif 60 <= rate_percentage < 75:
        return "iyi"
    else:
        return "çok iyi"

In [81]:
carlsen = carlsen.with_columns(
    pl.col("Best Move Rate").map_elements(classify_best_move_rate, return_dtype=pl.Utf8).alias("Best Move Rate Classify")
)

In [82]:
nakamura = nakamura.with_columns(
    pl.col("Best Move Rate").map_elements(classify_best_move_rate, return_dtype=pl.Utf8).alias("Best Move Rate Classify")
)

In [83]:
caruana = caruana.with_columns(
    pl.col("Best Move Rate").map_elements(classify_best_move_rate, return_dtype=pl.Utf8).alias("Best Move Rate Classify")
)

In [84]:
carlsen["Best Move Rate Classify"].value_counts()

Best Move Rate Classify,count
str,u32
"""iyi""",1607
"""kötü""",14
"""çok iyi""",53
"""orta""",3988


In [85]:
nakamura["Best Move Rate Classify"].value_counts()

Best Move Rate Classify,count
str,u32
"""çok iyi""",36
"""iyi""",1229
"""orta""",5997
"""kötü""",45


In [86]:
caruana["Best Move Rate Classify"].value_counts()

Best Move Rate Classify,count
str,u32
"""orta""",3450
"""çok iyi""",33
"""kötü""",12
"""iyi""",1093


## Oyun Bitme Nedeninin Sınıflandırılması

In [22]:
carlsen = carlsen.with_columns(
    pl.col('Mate').fill_null('0')
)

nakamura = carlsen.with_columns(
    pl.col('Mate').fill_null('0')
)

caruana = carlsen.with_columns(
    pl.col('Mate').fill_null('0')
)

In [87]:
def determine_game_end_reason(mate_value):
    if mate_value == "No forced mate detected":
        return "Resign"
    return "Mate"

In [88]:
carlsen = carlsen.with_columns(
    pl.col("Mate").map_elements(determine_game_end_reason, return_dtype=pl.Utf8).alias("Game Ending Reason")
)

In [89]:
nakamura = nakamura.with_columns(
    pl.col("Mate").map_elements(determine_game_end_reason, return_dtype=pl.Utf8).alias("Game Ending Reason")
)

In [90]:
caruana = caruana.with_columns(
    pl.col("Mate").map_elements(determine_game_end_reason, return_dtype=pl.Utf8).alias("Game Ending Reason")
)

In [91]:
carlsen["Game Ending Reason"].value_counts()

Game Ending Reason,count
str,u32
"""Mate""",813
"""Resign""",4849


In [97]:
carlsen.write_parquet("Carlsen V2.parquet")

In [98]:
nakamura.write_parquet("Nakamura V2.parquet")

In [99]:
caruana.write_parquet("Caruana V2.parquet")

In [35]:
carlsen["Game Class"].value_counts()

Game Class,count
str,u32
"""Smooth""",1004
"""Sharp""",395
"""Balanced""",969
"""Sudden""",3294
